In [34]:
import os
import random
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt



from MnistNet import MnistNet

In [35]:
NUM_EPOCHS = 6
BATCH_SIZE = 128
MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01
IMAGE_DIM = 28
NUM_CLASSES = 10
DEVICE_IDS = [0, 1, 2, 3]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
# print the seed value
# seed = torch.initial_seed()
# print('Used seed : {}'.format(seed))

In [37]:
mnistnet = MnistNet(num_classes=NUM_CLASSES).to(device)
mnistnet = torch.nn.parallel.DataParallel(mnistnet, device_ids=DEVICE_IDS)
print(mnistnet)
print('MnistNet created')

DataParallel(
  (module): MnistNet(
    (net): Sequential(
      (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(2, 2))
      (1): ReLU()
      (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): ReLU()
      (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU()
      (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=256, out_features=4096, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace

In [38]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(int)

In [39]:
print(X.shape)
X = X.reshape(X.shape[0], 1, 28, 28)
print(X.shape)

(70000, 784)
(70000, 1, 28, 28)


In [40]:
def attack_image(xs, img):
    img = img.reshape(28, 28)
    imgs = []
    for i, x in enumerate(xs):
            y_pos, x_pos, intensity = x
            imgs.append(img.copy())
            imgs[i][y_pos][x_pos] = intensity

    return imgs

In [41]:
X_attacked = []
y_attacked = []
for k, img in tqdm(enumerate(X)):
    img = img.reshape(28, 28)
    vec = []
    for i in range(5, 23, 4):
        for j in range(5, 23, 4):
            value = 255 if img[i][j] < 128 else 0
            pix = [i, j, value]
            vec.append(pix)

    attacked_imgs = attack_image(vec, img)
    X_attacked+=attacked_imgs
    y_attacked += [y[k]]*len(attacked_imgs)

X_attacked = np.array(X_attacked)
y_attacked = np.array(y_attacked)

70000it [00:22, 3061.84it/s]


In [42]:
print(len(X_attacked), len(y_attacked))

1750000 1750000


In [43]:
X_attacked = X_attacked.reshape(X_attacked.shape[0], 1, 28, 28)

In [44]:
TRAIN_PERCENTAGE = 0.8
TRAIN_NUMBER = int(len(X_attacked) * TRAIN_PERCENTAGE)

In [45]:
X_train, X_test, y_train, y_test = X_attacked[:TRAIN_NUMBER], X_attacked[TRAIN_NUMBER:], y_attacked[:TRAIN_NUMBER], y_attacked[TRAIN_NUMBER:]
train_dataset = TensorDataset(torch.from_numpy(X_train).float(),
                              torch.from_numpy(y_train).long())
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          pin_memory=True,
                          num_workers=8,
                          drop_last=True,
                          batch_size=BATCH_SIZE)

test_dataset = TensorDataset(torch.from_numpy(X_test).float(),
                             torch.from_numpy(y_test).long())
test_loader = DataLoader(test_dataset,
                          shuffle=True,
                          pin_memory=True,
                          num_workers=8,
                          drop_last=True,
                          batch_size=BATCH_SIZE)

In [46]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [47]:
def evaluate_model(nn_model):
    nn_model.eval()
    with torch.no_grad():
        correct_out = 0
        total_out = 0
        for pics, lbls in test_loader:
            out = nn_model(pics)
            pred = torch.argmax(out, dim=1)
            total_out += lbls.shape[0]
            correct_out += (pred == lbls).sum().item()

    loss_current = criterion(out, lbls)
    return correct_out / total_out, loss_current

In [48]:
# create optimizer
# the one that WORKS
optimizer = optim.Adam(params=mnistnet.parameters(), lr=0.0001)
### BELOW is the setting proposed by the original paper - which doesn't train....
# optimizer = optim.SGD(
#     params=alexnet.parameters(),
#     lr=LR_INIT,
#     momentum=MOMENTUM,
#     weight_decay=LR_DECAY)
print('Optimizer created')

Optimizer created


In [49]:
# multiply LR by 1 / 10 after every 30 epochs
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
print('LR Scheduler created')

accuracy_test_stat = []
accuracy_train_stat = []
loss_test_stat = []
loss_train_stat = []

# start training!!
print('Starting training...')
total_steps = 1
for epoch in range(NUM_EPOCHS):
    epoch_tr_accuracy = 0
    epoch_tr_loss = 0

    lr_scheduler.step()
    for imgs, classes in train_loader:
        imgs, classes = imgs.to(device), classes.to(device)

        # calculate the loss
        output = mnistnet(imgs)
        loss = F.cross_entropy(output, classes)

        # update the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # log the information and add to tensorboard
        if total_steps % 200 == 0:
            with torch.no_grad():
                predictions = torch.argmax(output, dim=1)
                accuracy = (predictions == classes).sum() / len(predictions)

                epoch_tr_accuracy = accuracy.item()
                epoch_tr_loss = loss.item()

                print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'
                      .format(epoch + 1, total_steps, loss.item(), epoch_tr_accuracy))



        # print out gradient values and parameter average values
        if total_steps % 400 == 0:
            with torch.no_grad():
                # print and save the grad of the parameters
                # also print and save parameter values
                print('*' * 10)
                for name, parameter in mnistnet.named_parameters():
                    if parameter.grad is not None:
                        avg_grad = torch.mean(parameter.grad)

                        # print('\t{} - grad_avg: {}'.format(name, avg_grad))
                        # tbwriter.add_scalar('grad_avg/{}'.format(name), avg_grad.item(), total_steps)
                        # tbwriter.add_histogram('grad/{}'.format(name),
                        #                       parameter.grad.cpu().numpy(), total_steps)
                    if parameter.data is not None:
                        avg_weight = torch.mean(parameter.data)

                        # print('\t{} - param_avg: {}'.format(name, avg_weight))
                        # tbwriter.add_histogram('weight/{}'.format(name),
                        #                       parameter.data.cpu().numpy(), total_steps)
                        # tbwriter.add_scalar('weight_avg/{}'.format(name), avg_weight.item(), total_steps)
        total_steps += 1

    # epoch_tr_accuracy /= len(train_loader)
    # epoch_tr_loss /= len(train_loader)

    a, l = evaluate_model(mnistnet)
    accuracy_train_stat.append(epoch_tr_accuracy)
    accuracy_test_stat.append(a)

    loss_train_stat.append(epoch_tr_loss)
    loss_test_stat.append(l)

    # print(accuracy_test_stat, accuracy_train_stat, loss_test_stat, loss_train_stat)



LR Scheduler created
Starting training...
Epoch: 1 	Step: 200 	Loss: 0.2748 	Acc: 0.921875
Epoch: 1 	Step: 400 	Loss: 0.1676 	Acc: 0.9609375
**********
Epoch: 1 	Step: 600 	Loss: 0.1376 	Acc: 0.9609375
Epoch: 1 	Step: 800 	Loss: 0.1124 	Acc: 0.9453125
**********
Epoch: 1 	Step: 1000 	Loss: 0.1092 	Acc: 0.953125
Epoch: 1 	Step: 1200 	Loss: 0.0669 	Acc: 0.984375
**********
Epoch: 1 	Step: 1400 	Loss: 0.0954 	Acc: 0.9609375
Epoch: 1 	Step: 1600 	Loss: 0.0131 	Acc: 1.0
**********
Epoch: 1 	Step: 1800 	Loss: 0.0263 	Acc: 0.9921875
Epoch: 1 	Step: 2000 	Loss: 0.0861 	Acc: 0.9609375
**********
Epoch: 1 	Step: 2200 	Loss: 0.0576 	Acc: 0.9765625
Epoch: 1 	Step: 2400 	Loss: 0.0405 	Acc: 0.9921875
**********
Epoch: 1 	Step: 2600 	Loss: 0.0111 	Acc: 0.9921875
Epoch: 1 	Step: 2800 	Loss: 0.0839 	Acc: 0.9921875
**********
Epoch: 1 	Step: 3000 	Loss: 0.0042 	Acc: 1.0
Epoch: 1 	Step: 3200 	Loss: 0.0046 	Acc: 1.0
**********
Epoch: 1 	Step: 3400 	Loss: 0.0211 	Acc: 0.9921875
Epoch: 1 	Step: 3600 	Loss: 

KeyboardInterrupt: 

In [ ]:
epochs = range(1, NUM_EPOCHS+1)
plt.figure(figsize=(10, 7))
plt.plot(epochs, loss_train_stat)
plt.plot(epochs, loss_test_stat)
plt.legend(["Train Loss", "Test Loss"])
plt.show()

In [ ]:
epochs = range(1, NUM_EPOCHS + 1)
plt.figure(figsize=(10, 7))
print(accuracy_train_stat)
plt.plot(epochs, accuracy_train_stat)
plt.plot(epochs, accuracy_test_stat)
plt.legend(["Train Accuracy", "Test Accuracy"])
plt.show()

In [ ]:
import cv2
import numpy as np

data = []
classes = [2, 3, 6, 8, 9, 1, 0, 4, 7, 5]
for i in range(10):
    img = cv2.imread("../my_dataset/"+str(i+1)+".png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    sharp_filter = np.array([[-1, 0, -1],
                             [ 0, 7,  0],
                             [-1, 0, -1]])
    img = cv2.filter2D(img, ddepth=-1, kernel=sharp_filter)

    img = cv2.bitwise_not(img)
    img = img.astype('float32')
    data.append(img)

data = np.reshape(data,(len(data), 1, 28, 28))
# data = np.array(data)
classes = np.array(classes)

print(data.shape)

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(data[i].reshape(28, 28), cmap='Greys')
    plt.title("Цифра %d" % classes[i])

In [ ]:
experiment_dataset = TensorDataset(torch.from_numpy(data).float(),
                                   torch.from_numpy(classes).long())
experiment_loader = DataLoader(experiment_dataset,
                          shuffle=False,
                          # pin_memory=True,
                          # num_workers=8,
                          # drop_last=True,

                          batch_size=BATCH_SIZE)

mnistnet.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in experiment_loader:
        outputs = mnistnet(images)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.shape[0]
        correct += (predicted == labels).sum().item()

    print("Test accurac"
          "y:",
        100 * correct / total, "%")


In [ ]:
plt.figure(figsize=(10, 7))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(data[i].reshape(28, 28), cmap='Greys')
    plt.title("%d, pred: %d" % (classes[i], predicted[i]))

In [ ]:
torch.save(mnistnet, 'mnistnet_pixel.pkl')

In [ ]:
mdl = torch.load('mnistnet_pixel.pkl')
mdl

In [ ]:
for images, labels in test_loader:
        outputs = mdl(images)
        predicted = torch.argmax(outputs, dim=1)

        print((predicted == labels).sum().item() / len(labels))